In [ ]:
# Collect Data
'''
Records screen from the game and saves it to a file
- GTA V 
- Windowed mode
- Resolution: 800 x 600
- Keep window at top left corner of screen (0,0)
'''

In [ ]:
# python 3.7.9
import numpy as np
print(np.__version__) #1.18.5

In [3]:
import os
import time
import cv2
import numpy as np
from grabscreen import grab_screen
from getkeys import key_check

key_map = {
    'W': [1, 0, 0, 0, 0, 0, 0, 0, 0],
    'S': [0, 1, 0, 0, 0, 0, 0, 0, 0],
    'A': [0, 0, 1, 0, 0, 0, 0, 0, 0],
    'D': [0, 0, 0, 1, 0, 0, 0, 0, 0],
    'WS': [0, 0, 0, 0, 1, 0, 0, 0, 0],
    'WD': [0, 0, 0, 0, 0, 1, 0, 0, 0],
    'SA': [0, 0, 0, 0, 0, 0, 1, 0, 0],
    'SD': [0, 0, 0, 0, 0, 0, 0, 1, 0],
    'NK': [0, 0, 0, 0, 0, 0, 0, 0, 1],
    'default': [0, 0, 0, 0, 0, 0, 0, 0, 0],
}
# key map for the output



starting_value = 1
BATCH_SIZE = 1000 #~370Mb/1000 images
mode = 'RGB' # 'RGB' or 'GRAY'

while True:
    file_name = 'training_data-{0}.npy'.format(starting_value)
    if os.path.isfile(file_name):
        print('File exists, moving along', starting_value)
        starting_value += 1
    else:
        print('File does not exist, starting fresh!', starting_value)
        break

def keys_to_output(keys):

    # Multi keeys issue
    '''
    Convert keys to a ...multi-hot... array
     0  1  2  3  4   5   6   7    8
    [W, S, A, D, WA, WD, SA, SD, NOKEY] boolean values.
    '''
    if ''.join(keys) in key_map:
        return key_map[''.join(keys)]
    return key_map['default']

def main(file_name, starting_value, mode, BATCH_SIZE):
    training_data = []
    for i in range(10, 0, -1):
        print(i + 1)
        time.sleep(1)
    paused = False
    print('STARTING!!!')
    while True:
        if not paused:
            screen = grab_screen(region=(0, 40, 800, 640))
            # region defines the area of the screen to capture

            screen = cv2.resize(screen, (480, 270))
            # resize the screen to 480x270

            mode = cv2.COLOR_BGR2RGB if mode == 'RGB' else cv2.COLOR_BGR2GRAY
            screen = cv2.cvtColor(screen, mode)
            # convert the screen from BGR to color mode

            keys = key_check()
            # get the keys pressed

            output = keys_to_output(keys)
            training_data.append([screen, output])
            # append the screen and the output to the training data

            if len(training_data) % 100 == 0:
                print(len(training_data))
                if len(training_data) == BATCH_SIZE:
                    np.save(file_name, np.array(training_data))
                    print('SAVED')
                    training_data = []
                    starting_value += 1
                    file_name = 'training_data-{0}.npy'.format(starting_value)

        keys = key_check()
        if 'T' in keys:
            if paused:
                paused = False
                print('Unpaused!')
                time.sleep(1)
            else:
                print('Pausing!')
                paused = True
                time.sleep(1)

if __name__ == "__main__":
    main(file_name, starting_value, mode, BATCH_SIZE)
    

File does not exist, starting fresh! 1
11
10
9
8
7
6
5
4
3
2
STARTING!!!
100
200
300
400
500
600
700
800
900
1000
SAVED
100
200
300
400
500
600
700
800
900
1000
SAVED
100
200
300
400


In [1]:
# balance_data.py

import numpy as np
import pandas as pd
from collections import Counter
from random import shuffle

train_data = np.load('training_data-1.npy', allow_pickle=True)  

df = pd.DataFrame(train_data)
#print(df.head())
print(Counter(df[1].apply(str)))

w = []
s =[]
a = []
d = []
wa = []
wd = []
sa = []
sd = []
nk = []

# 'W': [1, 0, 0, 0, 0, 0, 0, 0, 0],
# 'S': [0, 1, 0, 0, 0, 0, 0, 0, 0],
# 'A': [0, 0, 1, 0, 0, 0, 0, 0, 0],
# 'D': [0, 0, 0, 1, 0, 0, 0, 0, 0],
# 'WA': [0, 0, 0, 0, 1, 0, 0, 0, 0],
# 'WD': [0, 0, 0, 0, 0, 1, 0, 0, 0],
# 'SA': [0, 0, 0, 0, 0, 0, 1, 0, 0],
# 'SD': [0, 0, 0, 0, 0, 0, 0, 1, 0],
# 'NK': [0, 0, 0, 0, 0, 0, 0, 0, 1],
# 'default': [0, 0, 0, 0, 0, 0, 0, 0, 0],

for data in train_data:
    img = data[0]
    choice = data[1]

    if choice == [1, 0, 0, 0, 0, 0, 0, 0, 0]:
        w.append([img, choice])
    elif choice == [0, 1, 0, 0, 0, 0, 0, 0, 0]:
        s.append([img, choice])
    elif choice == [0, 0, 1, 0, 0, 0, 0, 0, 0]:
        a.append([img, choice])
    elif choice == [0, 0, 0, 1, 0, 0, 0, 0, 0]:
        d.append([img, choice])
    elif choice == [0, 0, 0, 0, 1, 0, 0, 0, 0]:
        wa.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 1, 0, 0, 0]:
        wd.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 0, 1, 0, 0]:
        sa.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 0, 0, 1, 0]:
        sd.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 0, 0, 0, 1]:
        nk.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 0, 0, 0, 0]:
        pass    
    else:
        print('no matches')    

print(len(w), len(s), len(a), len(d), len(wa), len(wd), len(sa), len(sd), len(nk))

#final_data = w + a + d
#shuffle(final_data)
#print(len(final_data))

#np.save('training_data_raw.npy', final_data)


# forwards = forwards[:len(lefts)][:len(rights)]
# lefts = lefts[:len(forwards)]
# rights = rights[:len(forwards)]

# final_data = forwards + lefts + rights
# shuffle(final_data)

# np.save('training_data.npy', final_data)





                                                   0  \
0  [[[47, 42, 38], [55, 48, 37], [51, 49, 39], [4...   
1  [[43, 46, 48, 49, 50, 63, 156, 199, 71, 46, 44...   
2  [[43, 46, 48, 49, 50, 63, 156, 199, 71, 46, 44...   
3  [[43, 46, 48, 49, 50, 63, 156, 199, 71, 46, 44...   
4  [[43, 46, 48, 49, 50, 63, 156, 199, 71, 46, 44...   

                             1  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0]  
1  [0, 0, 0, 0, 0, 0, 0, 0, 0]  
2  [0, 0, 0, 0, 0, 0, 0, 0, 0]  
3  [0, 0, 0, 0, 0, 0, 0, 0, 0]  
4  [0, 0, 0, 0, 0, 0, 0, 0, 0]  
Counter({'[0, 0, 0, 0, 0, 0, 0, 0, 0]': 640, '[1, 0, 0, 0, 0, 0, 0, 0, 0]': 284, '[0, 0, 0, 1, 0, 0, 0, 0, 0]': 49, '[0, 1, 0, 0, 0, 0, 0, 0, 0]': 21, '[0, 0, 1, 0, 0, 0, 0, 0, 0]': 6})
284 21 6 49 0 0 0 0 0


In [ ]:
# t_data = np.load('training_data_raw.npy', allow_pickle=True)